In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import re
from collections import Counter
import random

In [ ]:
model_name = "eryk-mazus/polka-1.1b-chat"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto"
).to(device)

streamer = TextStreamer(tokenizer, skip_prompt=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
def most_common_element(lst):
    counter = Counter(lst)
    most_common = counter.most_common(1)  # Get the most common element and its count
    return most_common[0][0] if most_common else "0"  # Return the element itself

In [ ]:
# Systemowy prompt
system_prompt = (
    "Jesteś matematykiem, który rozwiązuje równania. "
    "Nie mam problemu z matematyką, ale podaję ci równanie, a ty obliczasz wynik równania. "
    "Wynikami są liczby dodatnie i liczby ujemne (z minusem). "
    # "Jeśli wynik jest dodatni, to twoja odpowiedź powinna mieć format: Odpowiedź: liczba"
    # "Jeśli wynik jest ujemny, to twoja odpowiedź powinna mieć format: Odpowiedź: -liczba"
    # powyższy prompt daje bardzo złe wyniki
    "Odpowiadasz wyłącznie liczbą i niczym innym. Nie używasz liter ani słów."
)

system_prompt2 = (
    "Jesteś dobry z matematyki i podajesz wyniki równań. Daję Ci proste równania, a Ty podajesz mi wynik. Wynik jest tylko liczbą, nie generuj żadnych liter czy słów."
)
chat = [{"role": "system", "content": system_prompt2}]
chat.append({"role": "user", "content": "2+2="})
chat.append({"role": "assistant", "content": "Odpowiedź: 4"})
chat.append({"role": "user", "content": "3-6="})
chat.append({"role": "assistant", "content": "Odpowiedź: -3"})
chat.append({"role": "user", "content": "10-7="})
chat.append({"role": "assistant", "content": "Odpowiedź: 3"})
chat.append({"role": "user", "content": "7-10="})
chat.append({"role": "assistant", "content": "Odpowiedź: -3"})
chat.append({"role": "user", "content": "17-28="})
chat.append({"role": "assistant", "content": "Odpowiedź: -11"})
chat.append({"role": "user", "content": "42+55="})
chat.append({"role": "assistant", "content": "Odpowiedź: 97"})
chat.append({"role": "user", "content": "10*15="})
chat.append({"role": "assistant", "content": "Odpowiedź: 150"})
chat.append({"role": "user", "content": "4*7="})
chat.append({"role": "assistant", "content": "Odpowiedź: 28"})
chat.append({"role": "user", "content": "24/6 ="})
chat.append({"role": "assistant", "content": "Odpowiedź: 4"})

In [ ]:
def solve_equation(chat, equation):
    chat.append({"role": "user", "content": equation})
    inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_tensors="pt")
    first_param_device = next(model.parameters()).device
    inputs = inputs.to(first_param_device)
    responses = []
    for _ in range(3):
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                pad_token_id=tokenizer.eos_token_id,
                max_new_tokens=11,
                temperature=0.2,
                repetition_penalty=1.1,
                top_p=0.95,
                do_sample=True,
                num_return_sequences=1
            )
            new_tokens = outputs[0, inputs.size(1):]
            r = tokenizer.decode(new_tokens, skip_special_tokens=True)
            numeric_response = re.search(r'(\d+)', r)
            if numeric_response:
                responses.append(numeric_response.group(1))
            chat.append({"role": "assistant", "content": r})
    # print(responses)
    return int(most_common_element(responses))

In [ ]:
solve_equation(chat, "10*16=")

172

In [ ]:
def perform_equations(n = 20, equation_type = None):
    score = 0
    for i in range(n):
        operators = ["+", "-", "*"]
        e = ""
        if equation_type is None:
            e = f"{random.randint(1,15)}{random.choice(operators)}{random.randint(1,15)}"
        elif equation_type == "negative_result":
            a = random.randint(1,15)
            b = random.randint(1,15)
            if a > b:
                a, b = b, a
            e = f"{a}-{b}"
        elif equation_type == "simple_triple_addition":
            e = f"{random.randint(1,5)}+{random.randint(1,5)}+{random.randint(1,5)}"
        elif equation_type == "simple_addition":
            e = f"{random.randint(0,9)}+{random.randint(0,9)}"
        elif equation_type == "simple_addition_with_parenthesis":
            e = f"({random.randint(0,9)}+{random.randint(0,9)})"
        elif equation_type == "advanced_addition":
            e = f"{random.randint(100, 1000)}+{random.randint(100, 1000)}"
        elif equation_type == "simple_multiplication":
            e = f"{random.randint(0,9)}*{random.randint(0,9)}"
        elif equation_type == "simple_subtraction_negative":
            a = random.randint(0,9)
            b = random.randint(0,9)
            if a > b:
                a, b = b, a
            e = f"{a}-{b}"
        elif equation_type == "simple_subtraction_non_negative":
            a = random.randint(0,9)
            b = random.randint(0,9)
            if a < b:
                a, b = b, a
            e = f"{a}-{b}"
        elif equation_type == "division":
            a = random.randint(1,7)
            b = random.randint(1,7)
            e = f"{a*b}/{b}"
        r_model = solve_equation(chat, f"{e}=")
        r = eval(e)
        if r_model == r:
            score += 1
            print(f"GOOD, eq: {e}, model: {r_model}, answer: {r}, acc: {score/(i+1)}")
        else:
            print(f"BAD, eq: {e}, model: {r_model}, answer: {r}, acc: {score/(i+1)}")
    return score / n

In [ ]:
perform_equations(20)

GOOD, eq: 1+2, model: 3, answer: 3, acc: 1.0
GOOD, eq: 3*1, model: 3, answer: 3, acc: 1.0
GOOD, eq: 4+1, model: 5, answer: 5, acc: 1.0
GOOD, eq: 1*4, model: 4, answer: 4, acc: 1.0
GOOD, eq: 5*11, model: 55, answer: 55, acc: 1.0
BAD, eq: 15-6, model: 10, answer: 9, acc: 0.8333333333333334
BAD, eq: 14*8, model: 120, answer: 112, acc: 0.7142857142857143
GOOD, eq: 11+6, model: 17, answer: 17, acc: 0.75
BAD, eq: 9-10, model: 8, answer: -1, acc: 0.6666666666666666
GOOD, eq: 12*5, model: 60, answer: 60, acc: 0.7
BAD, eq: 4-1, model: 2, answer: 3, acc: 0.6363636363636364
GOOD, eq: 11+8, model: 19, answer: 19, acc: 0.6666666666666666
BAD, eq: 11*8, model: 100, answer: 88, acc: 0.6153846153846154
BAD, eq: 2-14, model: 6, answer: -12, acc: 0.5714285714285714
GOOD, eq: 6*2, model: 12, answer: 12, acc: 0.6
BAD, eq: 13*6, model: 112, answer: 78, acc: 0.5625
BAD, eq: 9-13, model: 13, answer: -4, acc: 0.5294117647058824
BAD, eq: 9*15, model: 90, answer: 135, acc: 0.5
BAD, eq: 11+12, model: 122, answer

0.45

In [ ]:
perform_equations(20, equation_type="negative_result")

BAD, eq: 8-13, model: 5, answer: -5, acc: 0.0
BAD, eq: 6-13, model: 3, answer: -7, acc: 0.0
BAD, eq: 7-12, model: 4, answer: -5, acc: 0.0
BAD, eq: 10-10, model: 10, answer: 0, acc: 0.0
BAD, eq: 1-6, model: 1, answer: -5, acc: 0.0
BAD, eq: 5-10, model: 5, answer: -5, acc: 0.0
BAD, eq: 6-15, model: 6, answer: -9, acc: 0.0
BAD, eq: 2-13, model: 2, answer: -11, acc: 0.0
BAD, eq: 10-11, model: 10, answer: -1, acc: 0.0
BAD, eq: 9-15, model: 9, answer: -6, acc: 0.0
BAD, eq: 10-15, model: 10, answer: -5, acc: 0.0
BAD, eq: 5-15, model: 5, answer: -10, acc: 0.0
BAD, eq: 4-13, model: 4, answer: -9, acc: 0.0
BAD, eq: 2-9, model: 2, answer: -7, acc: 0.0
BAD, eq: 10-15, model: 10, answer: -5, acc: 0.0
BAD, eq: 8-8, model: 8, answer: 0, acc: 0.0
BAD, eq: 14-14, model: 14, answer: 0, acc: 0.0
BAD, eq: 1-1, model: 1, answer: 0, acc: 0.0
BAD, eq: 2-8, model: 2, answer: -6, acc: 0.0
BAD, eq: 13-14, model: 13, answer: -1, acc: 0.0


0.0

In [ ]:
perform_equations(20, equation_type="simple_triple_addition")

BAD, eq: 1+2+4, model: 5, answer: 7, acc: 0.0
GOOD, eq: 4+2+1, model: 7, answer: 7, acc: 0.5
BAD, eq: 3+3+5, model: 7, answer: 11, acc: 0.3333333333333333
BAD, eq: 4+2+4, model: 8, answer: 10, acc: 0.25
BAD, eq: 2+4+3, model: 6, answer: 9, acc: 0.2
BAD, eq: 3+2+5, model: 6, answer: 10, acc: 0.16666666666666666
BAD, eq: 4+5+3, model: 7, answer: 12, acc: 0.14285714285714285
BAD, eq: 1+2+2, model: 3, answer: 5, acc: 0.125
BAD, eq: 1+3+2, model: 4, answer: 6, acc: 0.1111111111111111
BAD, eq: 4+5+5, model: 7, answer: 14, acc: 0.1
BAD, eq: 1+3+5, model: 4, answer: 9, acc: 0.09090909090909091
BAD, eq: 4+4+3, model: 7, answer: 11, acc: 0.08333333333333333
BAD, eq: 1+4+2, model: 4, answer: 7, acc: 0.07692307692307693
BAD, eq: 3+3+4, model: 7, answer: 10, acc: 0.07142857142857142
BAD, eq: 4+4+5, model: 7, answer: 13, acc: 0.06666666666666667
BAD, eq: 2+4+4, model: 4, answer: 10, acc: 0.0625
BAD, eq: 1+1+2, model: 2, answer: 4, acc: 0.058823529411764705
BAD, eq: 2+2+4, model: 4, answer: 8, acc: 0

0.05

In [ ]:
perform_equations(20, equation_type="simple_addition")

BAD, eq: 7+8, model: 14, answer: 15, acc: 0.0
GOOD, eq: 4+7, model: 11, answer: 11, acc: 0.5
GOOD, eq: 5+5, model: 10, answer: 10, acc: 0.6666666666666666
GOOD, eq: 4+8, model: 12, answer: 12, acc: 0.75
GOOD, eq: 7+6, model: 13, answer: 13, acc: 0.8
GOOD, eq: 8+5, model: 13, answer: 13, acc: 0.8333333333333334
GOOD, eq: 0+4, model: 4, answer: 4, acc: 0.8571428571428571
GOOD, eq: 3+4, model: 7, answer: 7, acc: 0.875
GOOD, eq: 3+6, model: 9, answer: 9, acc: 0.8888888888888888
GOOD, eq: 6+5, model: 11, answer: 11, acc: 0.9
GOOD, eq: 7+9, model: 16, answer: 16, acc: 0.9090909090909091
GOOD, eq: 4+3, model: 7, answer: 7, acc: 0.9166666666666666
GOOD, eq: 2+7, model: 9, answer: 9, acc: 0.9230769230769231
GOOD, eq: 7+5, model: 12, answer: 12, acc: 0.9285714285714286
GOOD, eq: 4+9, model: 13, answer: 13, acc: 0.9333333333333333
GOOD, eq: 2+3, model: 5, answer: 5, acc: 0.9375
GOOD, eq: 3+3, model: 6, answer: 6, acc: 0.9411764705882353
GOOD, eq: 5+0, model: 5, answer: 5, acc: 0.9444444444444444


0.95

In [ ]:
perform_equations(20, equation_type="simple_multiplication")

GOOD, eq: 6*4, model: 24, answer: 24, acc: 1.0
BAD, eq: 9*1, model: 99, answer: 9, acc: 0.5
GOOD, eq: 6*0, model: 0, answer: 0, acc: 0.6666666666666666
GOOD, eq: 3*7, model: 21, answer: 21, acc: 0.75
BAD, eq: 9*6, model: 99, answer: 54, acc: 0.6
BAD, eq: 4*8, model: 28, answer: 32, acc: 0.5
GOOD, eq: 7*7, model: 49, answer: 49, acc: 0.5714285714285714
GOOD, eq: 9*0, model: 0, answer: 0, acc: 0.625
GOOD, eq: 4*8, model: 32, answer: 32, acc: 0.6666666666666666
GOOD, eq: 2*6, model: 12, answer: 12, acc: 0.7
GOOD, eq: 4*2, model: 8, answer: 8, acc: 0.7272727272727273
GOOD, eq: 1*8, model: 8, answer: 8, acc: 0.75
GOOD, eq: 3*8, model: 24, answer: 24, acc: 0.7692307692307693
BAD, eq: 5*0, model: 5, answer: 0, acc: 0.7142857142857143
GOOD, eq: 7*0, model: 0, answer: 0, acc: 0.7333333333333333
BAD, eq: 8*1, model: 16, answer: 8, acc: 0.6875
GOOD, eq: 3*1, model: 3, answer: 3, acc: 0.7058823529411765
BAD, eq: 5*7, model: 25, answer: 35, acc: 0.6666666666666666
BAD, eq: 9*2, model: 128, answer: 

0.65

In [ ]:
perform_equations(20, equation_type="simple_subtraction_negative")

BAD, eq: 1-2, model: 1, answer: -1, acc: 0.0
BAD, eq: 2-7, model: 2, answer: -5, acc: 0.0
BAD, eq: 3-5, model: 3, answer: -2, acc: 0.0
BAD, eq: 0-7, model: 0, answer: -7, acc: 0.0
BAD, eq: 5-9, model: 5, answer: -4, acc: 0.0
BAD, eq: 5-6, model: 5, answer: -1, acc: 0.0
BAD, eq: 1-3, model: 1, answer: -2, acc: 0.0
BAD, eq: 0-4, model: 0, answer: -4, acc: 0.0
BAD, eq: 2-6, model: 2, answer: -4, acc: 0.0
BAD, eq: 0-6, model: 0, answer: -6, acc: 0.0
BAD, eq: 5-6, model: 5, answer: -1, acc: 0.0
BAD, eq: 1-5, model: 1, answer: -4, acc: 0.0
BAD, eq: 4-5, model: 4, answer: -1, acc: 0.0
BAD, eq: 5-6, model: 5, answer: -1, acc: 0.0
BAD, eq: 0-7, model: 0, answer: -7, acc: 0.0
BAD, eq: 7-9, model: 7, answer: -2, acc: 0.0
BAD, eq: 3-8, model: 3, answer: -5, acc: 0.0
BAD, eq: 1-6, model: 1, answer: -5, acc: 0.0
BAD, eq: 6-9, model: 6, answer: -3, acc: 0.0
BAD, eq: 2-6, model: 2, answer: -4, acc: 0.0


0.0

In [ ]:
perform_equations(20, equation_type="simple_subtraction_non_negative")

BAD, eq: 7-1, model: 7, answer: 6, acc: 0.0
BAD, eq: 6-1, model: 6, answer: 5, acc: 0.0
GOOD, eq: 7-4, model: 3, answer: 3, acc: 0.3333333333333333
GOOD, eq: 2-0, model: 2, answer: 2, acc: 0.5
BAD, eq: 9-4, model: 4, answer: 5, acc: 0.4
BAD, eq: 6-1, model: 6, answer: 5, acc: 0.3333333333333333
GOOD, eq: 2-0, model: 2, answer: 2, acc: 0.42857142857142855
GOOD, eq: 3-0, model: 3, answer: 3, acc: 0.5
BAD, eq: 7-5, model: 1, answer: 2, acc: 0.4444444444444444
BAD, eq: 9-7, model: 7, answer: 2, acc: 0.4
BAD, eq: 9-8, model: 8, answer: 1, acc: 0.36363636363636365
BAD, eq: 4-2, model: 4, answer: 2, acc: 0.3333333333333333
BAD, eq: 4-1, model: 4, answer: 3, acc: 0.3076923076923077
BAD, eq: 4-4, model: 4, answer: 0, acc: 0.2857142857142857
GOOD, eq: 3-0, model: 3, answer: 3, acc: 0.3333333333333333
GOOD, eq: 4-0, model: 4, answer: 4, acc: 0.375
BAD, eq: 6-6, model: 6, answer: 0, acc: 0.35294117647058826
BAD, eq: 7-1, model: 7, answer: 6, acc: 0.3333333333333333
BAD, eq: 7-4, model: 7, answer: 

0.3

In [ ]:
perform_equations(20, equation_type="division")

GOOD, eq: 18/6, model: 3, answer: 3.0, acc: 1.0
GOOD, eq: 12/2, model: 6, answer: 6.0, acc: 1.0
BAD, eq: 1/1, model: 0, answer: 1.0, acc: 0.6666666666666666
BAD, eq: 15/3, model: 4, answer: 5.0, acc: 0.5
BAD, eq: 30/6, model: 6, answer: 5.0, acc: 0.4
GOOD, eq: 30/5, model: 6, answer: 6.0, acc: 0.5
BAD, eq: 21/3, model: 6, answer: 7.0, acc: 0.42857142857142855
GOOD, eq: 16/4, model: 4, answer: 4.0, acc: 0.5
BAD, eq: 7/1, model: 1, answer: 7.0, acc: 0.4444444444444444
BAD, eq: 42/6, model: 6, answer: 7.0, acc: 0.4
BAD, eq: 12/3, model: 6, answer: 4.0, acc: 0.36363636363636365
BAD, eq: 2/1, model: 0, answer: 2.0, acc: 0.3333333333333333
BAD, eq: 28/4, model: 6, answer: 7.0, acc: 0.3076923076923077
BAD, eq: 21/3, model: 6, answer: 7.0, acc: 0.2857142857142857
BAD, eq: 12/3, model: 6, answer: 4.0, acc: 0.26666666666666666
GOOD, eq: 30/5, model: 6, answer: 6.0, acc: 0.3125
GOOD, eq: 8/2, model: 4, answer: 4.0, acc: 0.35294117647058826
GOOD, eq: 4/2, model: 2, answer: 2.0, acc: 0.388888888888

0.45

In [ ]:
perform_equations(20, equation_type="simple_addition_with_parenthesis")

GOOD, eq: (5+9), model: 14, answer: 14, acc: 1.0
GOOD, eq: (6+8), model: 14, answer: 14, acc: 1.0
GOOD, eq: (9+5), model: 14, answer: 14, acc: 1.0
GOOD, eq: (3+1), model: 4, answer: 4, acc: 1.0
GOOD, eq: (4+4), model: 8, answer: 8, acc: 1.0
GOOD, eq: (6+2), model: 8, answer: 8, acc: 1.0
BAD, eq: (6+5), model: 14, answer: 11, acc: 0.8571428571428571
BAD, eq: (2+9), model: 14, answer: 11, acc: 0.75
BAD, eq: (5+3), model: 14, answer: 8, acc: 0.6666666666666666
BAD, eq: (6+5), model: 14, answer: 11, acc: 0.6
BAD, eq: (5+0), model: 14, answer: 5, acc: 0.5454545454545454
BAD, eq: (5+2), model: 14, answer: 7, acc: 0.5
BAD, eq: (3+6), model: 14, answer: 9, acc: 0.46153846153846156
BAD, eq: (6+6), model: 14, answer: 12, acc: 0.42857142857142855
GOOD, eq: (3+3), model: 6, answer: 6, acc: 0.4666666666666667
GOOD, eq: (1+4), model: 5, answer: 5, acc: 0.5
BAD, eq: (9+3), model: 14, answer: 12, acc: 0.47058823529411764
BAD, eq: (6+3), model: 14, answer: 9, acc: 0.4444444444444444
BAD, eq: (8+7), mod

0.4

In [ ]:
perform_equations(20, equation_type="advanced_addition")

GOOD, eq: 735+248, model: 983, answer: 983, acc: 1.0
GOOD, eq: 625+690, model: 1315, answer: 1315, acc: 1.0
BAD, eq: 877+431, model: 1318, answer: 1308, acc: 0.6666666666666666
GOOD, eq: 727+312, model: 1039, answer: 1039, acc: 0.75
GOOD, eq: 769+512, model: 1281, answer: 1281, acc: 0.8
BAD, eq: 364+837, model: 4571, answer: 1201, acc: 0.6666666666666666
BAD, eq: 437+932, model: 5309, answer: 1369, acc: 0.5714285714285714
BAD, eq: 182+703, model: 2525, answer: 885, acc: 0.5
BAD, eq: 325+308, model: 6343, answer: 633, acc: 0.4444444444444444
BAD, eq: 277+786, model: 4553, answer: 1063, acc: 0.4
BAD, eq: 548+277, model: 7255, answer: 825, acc: 0.36363636363636365
BAD, eq: 253+268, model: 4211, answer: 521, acc: 0.3333333333333333
BAD, eq: 857+307, model: 1264, answer: 1164, acc: 0.3076923076923077
BAD, eq: 488+543, model: 9331, answer: 1031, acc: 0.2857142857142857
BAD, eq: 268+669, model: 9399, answer: 937, acc: 0.26666666666666666
BAD, eq: 854+386, model: 1241, answer: 1240, acc: 0.25


0.3

In [ ]:
# z dodawaniem sobie radzi całkiem dobrze
# z dowaniem liczb jednocyfrowych radzi sobie świetnie
# mnożenie jest zdecydowanie trudniejsze niż dodawanie
# fatalnie jest, kiedy wynikiem jest liczba ujemna
# model, który widział tylko przykłady z operacjami 2 liczb, słabo sobie radzi nawet z łatwymi przykładami, gdzie są 3 liczby
# model potrafi rozwiązać niektóre przykłady użycia operatora, którego nie miał w prompcie
# dodanie nawiasów obniża skuteczność modelu
# dodawanie dużych liczb jest znacząco trudniejsze